<a href="https://colab.research.google.com/github/dingjun6953/Accelerate-Python-codes-using-proper-libraries/blob/main/PySpark_and_Pandas_Dingjun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=2b0f0eebe4e9c34737c4ebf85f98713c69ff114d7419c14469da38685c087444
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [150]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PySpark_Pandas_Dingjun').getOrCreate()

Dataset  **all_stocks_5yr.csv** file could be downloaded from 
https://www.kaggle.com/datasets/camnugent/sandp500

# Load CSV in Pandas

In [151]:
%time
pd_df = pd.read_csv("all_stocks_5yr.csv")
pd_dataframe=pd.DataFrame(pd_df)
#pd_dataframe = pd_df.dropna()
pd_dataframe.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [116]:
pd_dataframe.describe()

,open,high,low,close,volume
count,619029.000000,619032.000000,619032.000000,619040.000000,6.190400e+05
mean,83.023334,83.778311,82.256096,83.043763,4.321823e+06
std,97.378769,98.207519,96.507421,97.389748,8.693610e+06
min,1.620000,1.690000,1.500000,1.590000,0.000000e+00
25%,40.220000,40.620000,39.830000,40.245000,1.070320e+06
50%,62.590000,63.150000,62.020000,62.620000,2.082094e+06
75%,94.370000,95.180000,93.540000,94.410000,4.284509e+06
max,2044.000000,2067.990000,2035.110000,2049.000000,6.182376e+08


In [121]:
pd_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619040 entries, 0 to 619039
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    619040 non-null  object 
 1   open    619029 non-null  float64
 2   high    619032 non-null  float64
 3   low     619032 non-null  float64
 4   close   619040 non-null  float64
 5   volume  619040 non-null  int64  
 6   Name    619040 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 33.1+ MB


# Load Data using Spark

In [152]:
%time
spark_dataframe = spark.read.csv("all_stocks_5yr.csv", header=True, inferSchema=True)
#spark_dataframe=spark_dataframe.na.drop(how="any")
print(spark_dataframe.show(5))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.63 µs
+-------------------+-----+-----+-----+-----+--------+----+
|               date| open| high|  low|close|  volume|Name|
+-------------------+-----+-----+-----+-----+--------+----+
|2013-02-08 00:00:00|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11 00:00:00|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12 00:00:00|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13 00:00:00| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14 00:00:00|14.94|14.96|13.16|13.99|31879900| AAL|
+-------------------+-----+-----+-----+-----+--------+----+
only showing top 5 rows

None


In [153]:
spark_dataframe.describe().show(truncate=True)

+-------+-----------------+-----------------+-----------------+-----------------+-----------------+------+
|summary|             open|             high|              low|            close|           volume|  Name|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+------+
|  count|           619029|           619032|           619032|           619040|           619040|619040|
|   mean|83.02333431454568|83.77831069346551|82.25609641375557|83.04376276476357|4321823.395568945|  null|
| stddev|97.37876904332367|98.20751890446478|96.50742105809123|97.38974800165921|8693609.511967609|  null|
|    min|             1.62|             1.69|              1.5|             1.59|                0|     A|
|    max|           2044.0|          2067.99|          2035.11|           2049.0|        618237630|   ZTS|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+------+



In [124]:
spark_dataframe.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)



## In Pandas

In [125]:
pd_dataframe[['open','close', 'volume']].corr()

,open,close,volume
open,1.000000,0.999872,-0.142705
close,0.999872,1.000000,-0.142802
volume,-0.142705,-0.142802,1.000000


## In Spark

In [126]:
from pyspark.sql.functions import corr
spark_dataframe.select(corr("open","volume")).show()

+--------------------+
|  corr(open, volume)|
+--------------------+
|-0.14270542442461023|
+--------------------+



## In Pandas

In [127]:
pd_dataframe.groupby('Name')['open','close'].min()

<ipython-input-127-c74650941c09>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd_dataframe.groupby('Name')['open','close'].min()


,open,close
Name,,
A,33.1800,33.3700
AAL,13.1400,13.0200
AAP,76.0400,76.1800
AAPL,55.4242,55.7899
ABBV,35.0500,35.2700
...,...,...
XYL,24.2000,24.1900
YUM,59.8900,59.7000
ZBH,72.5900,72.3800


## In Spark

In [128]:
from pyspark.sql import functions as F
spark_dataframe.groupBy("Name").agg(F.max("open"),F.max("close")).show()

+----+---------+----------+
|Name|max(open)|max(close)|
+----+---------+----------+
|ALXN|   206.66|    207.84|
| GIS|    72.65|     72.64|
|   K|     86.9|     86.98|
| LEN|    71.97|     71.82|
|SPGI|    183.8|     183.8|
| AIV|    47.82|     47.59|
| AVY|   121.43|    122.68|
|BF.B|    69.52|     69.58|
| MMM|   258.51|    258.63|
| PKI|    83.66|     82.75|
| PPG|   120.91|    121.47|
|  RF|    19.55|     19.54|
| AXP|   102.01|    101.22|
|  CI|   225.76|    226.22|
| IRM|     41.4|     41.44|
| WEC|    69.62|     69.53|
|INFO|    48.91|     48.87|
| PFG|    74.92|     75.04|
|  PM|   122.73|     122.9|
| SNA|   184.46|    183.76|
+----+---------+----------+
only showing top 20 rows



# Data Filtering

In [129]:
pd_dataframe.loc[pd_dataframe['Name'] == 'AAPL', ['open','close']].head()


,open,close
1259,67.7142,67.8542
1260,68.0714,68.5614
1261,68.5014,66.8428
1262,66.7442,66.7156
1263,66.3599,66.6556


In [130]:
spark_dataframe.filter("Name == 'GOOG'").select("open","close").show(5)

+-------+------+
|   open| close|
+-------+------+
|  568.0|558.46|
|  561.2|559.99|
| 566.89|556.97|
| 558.71|567.16|
|565.106| 567.0|
+-------+------+
only showing top 5 rows



### Column Selection

In [131]:
pdf=pd_dataframe[['Name', 'close']]
sdf=spark_dataframe.select('Name', 'close')

In [132]:
pdf.head()

,Name,close
0,AAL,14.75
1,AAL,14.46
2,AAL,14.27
3,AAL,14.66
4,AAL,13.99


In [133]:
sdf.show(5)

+----+-----+
|Name|close|
+----+-----+
| AAL|14.75|
| AAL|14.46|
| AAL|14.27|
| AAL|14.66|
| AAL|13.99|
+----+-----+
only showing top 5 rows



### In Pandas

In [138]:
#pd_dataframe['open'] =pd_dataframe['open'].astype(float)
#print(pd_dataframe.dtypes)
#pd_dataframe['open'] =pd.to_numeric(pd_dataframe['open'])
#pd_dataframe['open'] =pd. pd_dataframe['open']  .convert_objects(convert_numeric=True)

pd_dataframe['diff'] = pd_dataframe['close'] - pd.to_numeric(pd_dataframe['open'])
#pd_dataframe['close'] - pd.to_numeric(pd_dataframe['open'])


In [139]:
pd_dataframe.head()

,date,open,high,low,close,volume,Name,diff
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,-0.32
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,-0.43
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,-0.18
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,0.36
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,-0.95


### In Spark

In [140]:
from pyspark.sql.functions import col
spark_dataframe = spark_dataframe.withColumn("diff", col("close") - col("Open"))

In [141]:
spark_dataframe.show(5)

+-------------------+-----+-----+-----+-----+--------+----+--------------------+
|               date| open| high|  low|close|  volume|Name|                diff|
+-------------------+-----+-----+-----+-----+--------+----+--------------------+
|2013-02-08 00:00:00|15.07|15.12|14.63|14.75| 8407500| AAL| -0.3200000000000003|
|2013-02-11 00:00:00|14.89|15.01|14.26|14.46| 8882000| AAL| -0.4299999999999997|
|2013-02-12 00:00:00|14.45|14.51| 14.1|14.27| 8126000| AAL|-0.17999999999999972|
|2013-02-13 00:00:00| 14.3|14.94|14.25|14.66|10259500| AAL| 0.35999999999999943|
|2013-02-14 00:00:00|14.94|14.96|13.16|13.99|31879900| AAL| -0.9499999999999993|
+-------------------+-----+-----+-----+-----+--------+----+--------------------+
only showing top 5 rows



### In Pandas

In [142]:
pd_dataframe.head()

,date,open,high,low,close,volume,Name,diff
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,-0.32
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,-0.43
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,-0.18
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,0.36
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,-0.95


### In Spark

In [143]:
spark_dataframe.show(5)

+-------------------+-----+-----+-----+-----+--------+----+--------------------+
|               date| open| high|  low|close|  volume|Name|                diff|
+-------------------+-----+-----+-----+-----+--------+----+--------------------+
|2013-02-08 00:00:00|15.07|15.12|14.63|14.75| 8407500| AAL| -0.3200000000000003|
|2013-02-11 00:00:00|14.89|15.01|14.26|14.46| 8882000| AAL| -0.4299999999999997|
|2013-02-12 00:00:00|14.45|14.51| 14.1|14.27| 8126000| AAL|-0.17999999999999972|
|2013-02-13 00:00:00| 14.3|14.94|14.25|14.66|10259500| AAL| 0.35999999999999943|
|2013-02-14 00:00:00|14.94|14.96|13.16|13.99|31879900| AAL| -0.9499999999999993|
+-------------------+-----+-----+-----+-----+--------+----+--------------------+
only showing top 5 rows



#One Hot Encoding

###In Pandas

In [146]:
pd_dataframe_ohe = pd.get_dummies(pd_dataframe)

In [147]:
pd_dataframe_ohe.head()

,open,high,low,close,volume,diff,date_2013-02-08,date_2013-02-11,date_2013-02-12,date_2013-02-13,...,Name_XL,Name_XLNX,Name_XOM,Name_XRAY,Name_XRX,Name_XYL,Name_YUM,Name_ZBH,Name_ZION,Name_ZTS
0,15.07,15.12,14.63,14.75,8407500,-0.32,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14.89,15.01,14.26,14.46,8882000,-0.43,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14.45,14.51,14.10,14.27,8126000,-0.18,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,14.30,14.94,14.25,14.66,10259500,0.36,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,14.94,14.96,13.16,13.99,31879900,-0.95,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###In Spark


In [148]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType

categories = spark_dataframe.select('Name').distinct().rdd.flatMap(lambda x : x).collect()

categories.sort()

for category in categories:
    function = udf(lambda item: 1 if item == category else 0, IntegerType())
    new_column_name = 'Name'+'_'+category
    spark_dataframe = spark_dataframe.withColumn(new_column_name, function(col('Name')))

In [149]:
spark_dataframe.show()

+-------------------+-----+-----+-----+-----+--------+----+--------------------+------+--------+--------+---------+---------+--------+--------+--------+---------+--------+--------+--------+--------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+--------+---------+---------+---------+--------+--------+--------+---------+--------+--------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+--------+---------+---------+--------+---------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+---------+--------+---------+---------+-------+--------+--------+--------+----------+--------+--------+--------+------+-------+--------+--------+--------+-------+--------+---------+--------+--------+--------+---------+---------+---------+-------+--------+--------+--------+---------+---------+-------+---------+